In [ ]:
# convert model tf.__version__>=2.2.0
import tensorflow as tf
model = tf.keras.models.load_model("./model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [19]:
# load tflite model as interpreter
import numpy as np
import tensorflow as tf
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# model.summary()
input_details = interpreter.get_input_details()
print(input_details, '\n')
output_details = interpreter.get_output_details()
print(output_details, '\n')

# input data
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
print(input_data, '\n')
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[{'name': 'input_1', 'index': 0, 'shape': array([ 1, 40]), 'shape_signature': array([ 1, 40]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}] 

[{'name': 'Identity', 'index': 18, 'shape': array([ 1, 40]), 'shape_signature': array([ 1, 40]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}] 

[[0.68465936 0.49161127 0.8062872  0.5963613  0.31112885 0.18036556
  0.60514456 0.5218083  0.25737634 0.28824732 0.75324917 0.9015781
  0.77119833 0.92593616 0.50278217 0.5187445  0.43480816 0.90550584
  0.7134789  0.6246327  0.57040125 0.93511176 0.34488136 0.2572808
  0.6458222  0.40008554 0.9892506  0.49363065 0.49609613 0.47795162
  0.22490299 0.48104107 0.4297

In [44]:
import tensorflow as tf
model = tf.keras.models.load_model("model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
    for _ in range(100):
        yield [[[np.float32(np.random.randint(1,100)) for i in range(40)]]] #, np.int8
    # Get sample input data as a numpy array in a method of your choosing.
        #yield [input]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()
with open('model.tflite', 'wb') as f:
    f.write(tflite_quant_model)

In [45]:
# load tflite model as interpreter
import numpy as np
import tensorflow as tf
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# model.summary()
input_details = interpreter.get_input_details()
print(input_details, '\n')
output_details = interpreter.get_output_details()
print(output_details, '\n')

# input data
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
print(input_data, '\n')
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[{'name': 'input_1', 'index': 19, 'shape': array([ 1, 40]), 'shape_signature': array([ 1, 40]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}] 

[{'name': 'Identity', 'index': 20, 'shape': array([ 1, 40]), 'shape_signature': array([ 1, 40]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}] 

[[0.11475182 0.55889523 0.48353592 0.9742087  0.3655403  0.33972397
  0.5673594  0.51464975 0.9185434  0.96213394 0.10205621 0.21504642
  0.57998663 0.45143706 0.85620815 0.5110599  0.73642844 0.74053794
  0.8430483  0.7141178  0.1547634  0.39900663 0.5444536  0.62679064
  0.75079167 0.22589648 0.58749515 0.8883525  0.16994989 0.78576547
  0.10739439 0.335771   0.4